In [ ]:
"""
Author: Vineet Madan
Purpose: Lab Assignment 2 Part 2 of CS529 Applied Artificial Intelligence
Date: 8 October 2019
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import json
import random
from sklearn.decomposition import TruncatedSVD
import csv
from nltk.stem import PorterStemmer 
np.random.seed(0)

In [ ]:
DATA_FILE = open('nytimes_news_articles.txt', 'r')
DATA_LINES = DATA_FILE.readlines()

In [ ]:
DATA_URLS = []
DATA_TEXT = []

for line in DATA_LINES:
    if line.startswith('URL'):
        DATA_URLS.append(line)
        DATA_TEXT.append([])
    else:
        DATA_TEXT[-1].append(line)
        
#clean the text a bit
for txt in DATA_TEXT:
    try:
        while True:
            txt.remove('\n')
    except:
        pass
    try:
        while True:
            txt.remove('')
    except:
        pass
    try:
        while True:
            txt.remove(' ')
    except:
        pass
    try:
        while True:
            txt.remove('\t')
    except:
        pass
    
#     print(txt)

In [ ]:
DATA_URLS = [url[27: len(url)-6] for url in DATA_URLS]

In [ ]:
"""
Q1. starts here
"""

tknzd_urls = []
categories = []
subcategories = []
for url in DATA_URLS:
    temp = url.split('/')
    if temp[-3].isnumeric()==True:
        category = temp[-2]
        headline = temp[-1]
        headline= headline.replace('-', ' ')
        subcategory = ''
        if category not in categories:#add only if unique
            categories.append(category)
    else:
        category = temp[-3]
        subcategory = temp[-2]
        headline = temp[-1]
        headline= headline.replace('-', ' ')
        if category not in categories:#add only if unique
            categories.append(category)
        if subcategory not in subcategories:#add only if unique
            subcategories.append(subcategory)
        
    tknzd_urls.append([category, subcategory, headline])

In [ ]:
tknzd_urls

In [ ]:
with open('nytimes_processed_dset.csv', mode='w') as csv_file:
    url_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for tkn_url, content in zip(tknzd_urls, DATA_TEXT):
            url_writer.writerow(tkn_url+[content])

In [ ]:
print("Number of unique  categories = {}; Number of unique subcategories = {};".format(len(categories), len(subcategories)))
"""
Q1 ends here
"""

In [ ]:
DATA_TEXT = DATA_TEXT[:500]
len(DATA_TEXT)
NUM_DOCS=500#reduce the size of the dataset to comply with hardware realities

In [ ]:
###
#construct the word corpus
###

from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

CORPUS = []
WORDS_IN_DOCS = []

for lines in DATA_TEXT:
    WORDS_IN_DOCS.append([])
    for line in lines:
        words = line.split()
        for word in words:
            try:
                while True:
                    word.remove('"')
            except:
                pass
            try:
                while True:
                    word.remove(' ')
            except:
                pass
            try:
                while True:
                    word.remove('\'')
            except:
                pass
            
            word = ''.join(e for e in word if e.isalpha())
            if word not in stop_words and word!='':
                CORPUS.append(word)
                WORDS_IN_DOCS[-1].append(word)
            

In [ ]:
print(len(CORPUS))

In [ ]:
### do stemming

#dict whose key is stem and value is the corresponding word
stems = {}

ps = PorterStemmer()
for w_id in range(len(CORPUS)): 
    if w_id%2000000==0:
        print(w_id)
    stem = ps.stem(CORPUS[w_id])
    if stem not in stems.keys():
        stems[stem] = [CORPUS[w_id].lower(), 1]
    else:
        stems[stem][1] = stems[stem][1]+1
     

In [ ]:
NUM_WORDS = len(stems)
# NUM_DOCS = len(DATA_URLS)

WORD_VEC_MAT = np.zeros(shape=(NUM_DOCS, NUM_WORDS), dtype='uint16')

In [ ]:
NUM_DOCS, NUM_WORDS

In [ ]:
def get_w_id_from_word(stems, word):
    i = 0
    for key in stems.keys():
        if key.lower()==word.lower():
            return i
        i=i+1
    return None

In [ ]:
#fill the word vector matrix by iterating through the WORDS_IN_DOCS
load_word_vec_mat_from_file = True

if load_word_vec_mat_from_file==True:
    WORD_VEC_MAT=np.load('word_vec_model_red_red.npy')
else:
    WORD_VEC_MAT.fill(0)
    for doc_id in range(NUM_DOCS):
        print(doc_id)
        for w_index in range(len(WORDS_IN_DOCS[doc_id])):
            st = ps.stem(WORDS_IN_DOCS[doc_id][w_index])
            w_id = get_w_id_from_word(stems, st)
            WORD_VEC_MAT[doc_id][w_id] = WORD_VEC_MAT[doc_id][w_id] +1

In [ ]:
#save the WORD_VEC_MAT to file...its a np array
if not load_word_vec_mat_from_file:
    np.save('word_vec_model_red_red.npy', WORD_VEC_MAT)

In [ ]:
WORD_VEC_MAT = WORD_VEC_MAT.astype('float')

In [ ]:
#now get the IDF for each of the words...as a numpy array of size [NUM_WORDS] and save it to file
#find the idf for each word
WORD_DOC_FREQS = np.zeros(NUM_WORDS)

load_word_doc_freqs_from_file = True

if load_word_doc_freqs_from_file==True:
    WORD_DOC_FREQS = np.load('word_doc_freqs_red_red.npy')
else:
    for w_id in range(NUM_WORDS):
        if w_id%10==0:
            print(w_id)
        freq = 0
        for doc_id in range(NUM_DOCS):
            if WORD_VEC_MAT[doc_id][w_id] > 0:
                freq+=1
        WORD_DOC_FREQS[w_id] = freq

In [ ]:
if not load_word_doc_freqs_from_file:
    np.save('word_doc_freqs_red_red.npy', WORD_DOC_FREQS)

In [ ]:
INV_DOC_FREQS = np.zeros(NUM_WORDS)
INV_DOC_FREQS.fill(NUM_DOCS)
for ith in range(INV_DOC_FREQS.shape[0]):
    if not WORD_DOC_FREQS[ith]==0:
        INV_DOC_FREQS[ith] = np.log(NUM_DOCS/WORD_DOC_FREQS[ith])

"""
The final tfidf marix is defined here
"""
FINAL_TFIDF_MATRIX = np.copy(WORD_VEC_MAT)
for doc_id in range(NUM_DOCS):
    FINAL_TFIDF_MATRIX[doc_id] = np.multiply(WORD_VEC_MAT[doc_id], INV_DOC_FREQS)

In [ ]:
"""
Q3. LSA
"""
#convert a 1d matrix of singluar values returned by the svd function to a 2d square matrix
def convert_singluar_to_diagonal(mat):
    sz = mat.shape[0]
    res = np.zeros(shape=(sz, sz))
    for i in range(sz):
        res[i][i] = mat[i]
    return res

#the input is the word_to_vector matrix after applying tfidf
def do_lsa(final_tfidf_mat, k=100):
    num_docs = final_tfidf_mat.shape[0]
    num_terms = final_tfidf_mat.shape[1]
    
    u,s,v = np.linalg.svd(final_tfidf_mat)
    
    s = convert_singluar_to_diagonal(s[:k])
    
    u,s,v = np.linalg.svd(final_tfidf_mat)
    s = np.reshape(s, (s.shape[0], 1))

    s = convert_singluar_to_diagonal(s)

    #get the top-k eigenvals etc.
    u = u[:, :k]
    v = v[:, :k]
    s = s[:k, :k]
    
    return u,s,v

In [ ]:
u,s,v = do_lsa(final_tfidf_mat=FINAL_TFIDF_MATRIX, k=100)

In [ ]:
"""
Q4. PLSA begins here
"""
